# Convert foreign payments to USD with historical FX rates

In [1]:
import os
import pandas as pd

from expat_fatcat import expat_fatcat as fatcat

from importlib import reload
reload(fatcat);


# Load example salary data

This data is fake. Really.

In [2]:
salary = pd.read_csv('example.csv')
salary

,Date,Amount
0,2017-01-20,1000
1,2017-02-20,1000
2,2017-03-20,1500


## Get smoothed historical exchange rates to USD via quandl

The workhorse of this package is the function `get_fx_rate_smoothed` that calls the quandl api to get the exchange rate for each payment in your salary, rental payment, etc.. One edge case handled by `get_fx_rate_smoothed` is when you have a payment on a day that is a bank holiday, which results in `NaN`. The solution implemented here is to look for the first valid fx-rate before and after the day you were payed and average them.

In [3]:
date_string=salary['Date'][0]
fx_call_code = fatcat.quandl_calls['EUR']['ECB']
print(f'EUR to USD on {date_string}: {fatcat.get_fx_rate_smoothed(date_string, fx_call_code)}')

fx_call_code = fatcat.quandl_calls['GBP']['BOE']
print(f'GBP to USD on {date_string}: {fatcat.get_fx_rate_smoothed(date_string, fx_call_code)}')

EUR to USD on 2017-01-20: 1.0632
GBP to USD on 2017-01-20: 1.2323


The above exchange rates were sourced from the Bank of England (BOE). Let's see what happens if we take our GBP to USD exchange from the Federal Reserve (Federal Research Economic Data) rather than the BOE.

In [4]:
fx_call_code = fatcat.quandl_calls['GBP']['FRED']
print(f'GBP to USD on {date_string} from the Federal Reserve, USA:'
      f'{fatcat.get_fx_rate_smoothed(date_string, fx_call_code)}')

GBP to USD on 2017-01-20 from the Federal Reserve, USA:1.2402


## Convert a series of payment to USD

Accepted input formats for `expat_fatcat` are csv and xlsx.

In [5]:
fatcat.convert_payments('example.csv')

,Date,Amount,fx_rate,amount_usd
0,2017-01-20,1000,1.0632,1063.2
1,2017-02-20,1000,1.0616,1061.6
2,2017-03-20,1500,1.0752,1612.8


In [6]:
fatcat.convert_payments('example.xlsx')

,Date,Amount,fx_rate,amount_usd
0,2017-01-20,1000,1.0632,1063.2
1,2017-02-20,1000,1.0616,1061.6
2,2017-03-20,1500,1.0752,1612.8


## Compute total foreign income  and housing deduction examples of f2555

In [7]:
f2555_line_25 = fatcat.sum_payments_usd('example_salary.csv')
print(f'Example salary calculation, line 25 of f2555: {f2555_line_25}')

f2555_line_34 = fatcat.sum_payments_usd('example_housing.csv')
print(f'Example housing deduction calculation, line 34 of f2555: {f2555_line_34}')


Example salary calculation, line 25 of f2555: 3737.6000000000004
Example housing deduction calculation, line 34 of f2555: 2080.0
